# Tutorial PXCT data analysis (PART 1) - HERCULES school 2021

### Tutor: Julio C. da Silva (Néel Institute CNRS, Grenoble, France) 
### email: julio-cesar.da-silva@neel.cnrs.fr
#### Personal webpage: https://sites.google.com/view/jcesardasilva

### <span style="color:red">** Disclaimer: This notebook is intended from educational reasons only.**</span>

<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://mybinder.org/v2/gh/jcesardasilva/tutorialHercules.git/HEAD"><img src="https://mybinder.org/static/images/logo_social.png" height="52" width="62"/>Run in MyBinder.org</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer" href="https://colab.research.google.com/github/jcesardasilva/tutorialHercules/blob/master/PXCT/PXCT_pipeline_part1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://jupyter-slurm.esrf.fr/"><img src="https://upload.wikimedia.org/wikipedia/fr/thumb/3/37/ESRF_-_Grenoble.png/280px-ESRF_-_Grenoble.png" height="40" width="40"/>Run in Jupyter-Slurm at ESRF (needs login)</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer" href="https://github.com/jcesardasilva/tutorialHercules.git"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
   </td>
    
</table>

**All the steps of this processing pipeline are described in:**

*da Silva, J. C., Haubrich, J., Requena, G., Hubert, M., Pacureanu, A., Bloch, L., Yang, Y., Cloetens, P., “High energy near-and far-field ptychographic tomography at the ESRF”. Proc. SPIE 10391, Developments in X-Ray Tomography XI, 1039106 (2017). [doi: 10.1117/12.2272971](https://doi.org/10.1117/12.2272971)*

**REMARKS: the references of this paper contains a list of important and more pioneering works on the PXCT data processing steps**

### Toupy - Tomographic Utilities for Python

<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://toupy.readthedocs.io"><img src="https://toupy.readthedocs.io/en/latest/_static/toupy_logo.png" width="20%"/></a>
  </td>
</table>

**For the PXCT data analysis, we will use the python suite [*Toupy*](https://toupy.readthedocs.io/), which is maintained by me, but open to contributions from the community.**

<span style="color:blue">**Another suite of routines for PXCT data processing is [*PtychoShelves*](https://journals.iucr.org/j/issues/2020/02/00/zy5001/), which is high optimized and provided even very advanced algorithms. It run in MATLAB and it is compatible with GPUs**</span>

This tutorial needs a package called Toupy, which can be installed as:

In [ ]:
#!pip3 install --user --force-reinstall --upgrade git+https://github.com/jcesardasilva/toupy.git # uncomment this line if not installed
#!pip3 install --user --upgrade ipympl

Importing the dependencies:

In [ ]:
%matplotlib ipympl
# standard packages
import warnings
warnings.filterwarnings('ignore') # getting rid of annoying warnings.
# third party packages
import ipympl
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import toupy

#### Loading the data
We now load the data. We will use a dataset we obtained at cSAXS beamline (PSI, CH) by applying PXCT on the characterizationn of a shaped Fluid Catalytic Cracking (FCC) catalyst. The open-access publication from 2015 can be found [here](https://doi.org/10.1002/cctc.201402925)*. FCC catalysts are important shaped catalysts for the oil industry which enable the conversion of crude oil (petroleum) into more valuable products (E.g. gasoline, olefinic gases).

***da Silva, J. C., Mader, K., Holler, M., Haberthür, D., Diaz, A., Guizar-Sicairos, M., Cheng, W-.C., Shu, Y., Raabe, J., Menzel, A., van Bokhoven, J. A., “Assessment of the 3D pore structure and individual components of preshaped catalyst bodies by X-ray imaging”, ChemCatChem 7, 413-416 (2015)**

The file containing the data should be `PXCTprojections.npz`, which contains the complex-values PXCT projections. Let us download the file from internet first:

In [ ]:
# Toupy provides a routine to download the data from the URL
from toupy.utils import downloadURLfile
import tarfile
tarfname = "PXCTprojections.tar.gz"
urldata = 'https://cloud.neel.cnrs.fr/index.php/s/kjXWNMzsYEzYa34/download'
downloadURLfile(urldata, 'PXCTprojections.npz') # this may take time depending on internet speed
# uncompress and extract the file
tf = tarfile.open(tarfname)
tf.extractall()

<span style="color:red"> **Remark:**
For the sake of speed and to reduce the computing resources required for this tutorial, the projections have been downsized (binned) by a factor 2, which also means the pixel size is twice larger than the original pixel size. Nevertheless, this does not affect the data processing step, which should be identical for the original dataset. </span>

We can now load the data from the file:

In [ ]:
fname = "PXCTprojections.npz"
data_dict = np.load(fname) # load the file
list(data_dict.files) # this one list the keys of the data dictionary extracted from the file

You can, for example, get the wavelength of the pixel size:

In [ ]:
wavelen = data_dict['wavelen']
pixsize = data_dict['psize']

print(f"The projection pixel size of the projections is {pixsize/1e-9:.02f} nm")
print(f"The wavelenth of the incoming photons is {wavelen/1e-10:.02f} Angstroms")

We should also get the tomographic angles (`theta`) and the complex-valued projections, from which we should extract only the phases:

In [ ]:
theta = data_dict['theta']
projections = data_dict['projections'] # <- ATTENTION: this one is memory consuming. 
projections = np.angle(projections) # Get the phases of the complex-valued projections
nproj, nr, nc = projections.shape

In [ ]:
delta_theta = np.diff(theta)
print(f"The angular sampling interval is {delta_theta[0]:.02f} degrees")

Let us take a look at one projection. I will select the first and last ones,i.e. at angles 0 and 180-$\Delta\theta$ degress:

In [ ]:
plt.close('all')
fig1 = plt.figure(1,figsize=(10,4))
ax1 = fig1.add_subplot(121)
im1 = ax1.imshow(projections[0],cmap='bone', vmin=-4, vmax=1)
ax1.set_title('Phase proj. at 0 degrees',fontsize = 14)
#cax = divider.append_axes('right', size='5%', pad=0.05)
#fig1.colorbar(im1,cax=cax)
ax2 = fig1.add_subplot(122)
im2 = ax2.imshow(projections[-1],cmap='bone', vmin=-4, vmax=1)
ax2.set_title('Phase proj. at (180-3.2) degrees',fontsize = 14)
display.display(fig1)
#display.display(fig1.canvas)
#display.clear_output(wait=True)

#### Removing linear phase ramp
You may have notice the linear phase ramp in the image. The first step of the PXCT data analysis is correcting for that ramp as we do below. 

**Toupy** provides an iterative enviroment to select the mask area, but interactive plot from matplotlib are not supported in Jupyter notebooks, unfortunately. Otherwise, one could use the commented lines below:

In [ ]:
#from toupy.restoration import gui_plotphase
#stack_phasecorr = gui_plotphase(projections,**params) #<-- iterative plots like this are not allowed in jupyter, 
                                                      #  but can be used out of the jupyter enviroment

So, instead of the interactive plot, let us do it manually. Let us first plot one projection to look at the ramp and air area:

In [ ]:
import matplotlib.gridspec as gridspec
#-----------
# parameters
#------------
projnum = 1 # projection number
pvmin = -2.1 # gray scale minimum value
pvmax = 2 # gray scale maximum value
#------------
# Displaying the images
fig2 = plt.figure(2,figsize=(6,8))
gs = gridspec.GridSpec(
        2, 1, width_ratios=[4], height_ratios=[7,3]  # figure=4,
    )
ax1 = plt.subplot(gs[0])
im1 = ax1.imshow(projections[projnum],cmap="bone",vmin=pvmin,vmax=pvmax)
ax1.axis('tight')
ax1.plot([0,nc],[int(nr/2),int(nr/2)],'r--')
ax2 = plt.subplot(gs[1])
im2 = ax2.plot(projections[projnum,np.int(nr/2),:])
ax2.plot([0,nc],[0,0])
ax2.set_xlim([0,nc])
ax2.set_ylim([pvmin,pvmax])
display.display(fig2)
display.display(fig2.canvas)
display.clear_output(wait=True)

#### Creating the mask
We should create a mask containing only area with air/vacuum around the sample:

In [ ]:
maskair = np.zeros_like(projections[0])
maskair[100:400,70:150]=1
maskair[100:400,490:550]=1
#-----------
# parameters
#------------
projnum = 1 # projection number
pvmin = -2.1 # gray scale minimum value
pvmax = 2 # gray scale maximum value
#------------

# Displaying the images
fig2 = plt.figure(2,figsize=(6,8))
gs = gridspec.GridSpec(
        2, 1, width_ratios=[4], height_ratios=[7,3]  # figure=4,
    )
ax1 = plt.subplot(gs[0])
im1 = ax1.imshow(projections[projnum]+maskair,cmap="bone",vmin=pvmin,vmax=pvmax)
ax1.axis('tight')
ax1.plot([0,nc],[int(nr/2),int(nr/2)],'r--')
ax2 = plt.subplot(gs[1])
im2 = ax2.plot(projections[projnum,np.int(nr/2),:])
ax2.plot([0,nc],[0,0])
ax2.set_xlim([0,nc])
ax2.set_ylim([pvmin,pvmax])
display.display(fig2)
display.display(fig2.canvas)
display.clear_output(wait=True)

#### Applying the mask
Let us apply the mask to one projection

In [ ]:
from toupy.restoration import rmlinearphase
from toupy.utils import progbar

In [ ]:
#-----------
# parameters
#------------
projnum = 1 # projection number
pvmin = -3 # gray scale minimum value
pvmax = 1 # gray scale maximum value
#------------
imgin = np.exp(1j * projections[projnum]).copy() # Deep copy
projcorr = np.angle(rmlinearphase(imgin, maskair)).copy()
# Displaying the images again
fig2 = plt.figure(2,figsize=(6,8))
gs = gridspec.GridSpec(
        2, 1, width_ratios=[4], height_ratios=[7,3]  # figure=4,
    )
ax1 = plt.subplot(gs[0])
im1 = ax1.imshow(projcorr+maskair,cmap="bone",vmin=pvmin,vmax=pvmax)
ax1.axis('tight')
ax1.plot([0,nc],[int(nr/2),int(nr/2)],'r--')
ax2 = plt.subplot(gs[1])
im2 = ax2.plot(projcorr[np.int(nr/2),:])
ax2.plot([0,nc],[0,0])
ax2.set_xlim([0,nc])
ax2.set_ylim([pvmin,pvmax])
display.display(fig2)
display.display(fig2.canvas)
display.clear_output(wait=True)

Great! It works. So, we can proceed with all the other projections:

In [ ]:
projcorr = np.empty_like(projections)
for ii in range(nproj):
    strbar = "Projection {} out of {}".format(ii + 1, nproj)
    imgin = np.exp(1j * projections[ii]).copy() # Deep copy
    projcorr[ii] = np.angle(rmlinearphase(imgin, maskair)).copy()
    progbar(ii+1,nproj,strbar)

In [ ]:
# Do you want to check the results? Take a look:
#-----------
# parameters
#------------
projnum = 200 # projection number
pvmin = -3 # gray scale minimum value
pvmax = 1 # gray scale maximum value
#------------

# Displaying the images
fig2 = plt.figure(2,figsize=(6,8))
gs = gridspec.GridSpec(
        2, 1, width_ratios=[4], height_ratios=[7,3]  # figure=4,
    )
ax1 = plt.subplot(gs[0])
im1 = ax1.imshow(projcorr[projnum],cmap="bone",vmin=pvmin,vmax=pvmax)
ax1.axis('tight')
ax1.plot([0,nc],[int(nr/2),int(nr/2)],'r--')
ax2 = plt.subplot(gs[1])
im2 = ax2.plot(projcorr[projnum,np.int(nr/2),:])
ax2.plot([0,nc],[0,0])
ax2.set_xlim([0,nc])
ax2.set_ylim([pvmin,pvmax])
display.display(fig2)
display.display(fig2.canvas)
display.clear_output(wait=True)

#### Phase unwrapping
Although there are very few area where the phases are wrapped, it will be benefitial to unwrap them.

In [ ]:
# import the toupy routines we will need
from toupy.restoration import phaseresidues, chooseregiontounwrap, unwrapping_phase
from toupy.utils import iterative_show

In [ ]:
# find the residues and choose region to be unwrapped
rx, ry, airpix = chooseregiontounwrap(projcorr)

In [ ]:
projunwrap = unwrapping_phase(projcorr, rx, ry, airpix, vmin=pvmin, vmax=pvmax)

In [ ]:
del projcorr
del projections # release some memory

#### Let us save our progress so far and make a break for discution/questions

In [ ]:
outputfname = "PXCTcorrprojections.npz"
np.savez(outputfname, wavelen = wavelen, psize = pixsize, projections = projunwrap, theta = theta)

In [ ]:
!ls -lrth